In [6]:
del list

NameError: name 'list' is not defined

In [1]:
# Tian 1:10, 17:26
subcortical_index = list(range(0,10)) + list(range(16,26))
# Schaefer: 
# lh-mPFC: 199:205
# rh-mPFC: 464:470
# lh-Ins: 67, 108:111, 126:128
# rh-Ins: 319, 361:364, 383:386
## ACC: 390
# Glasser
cortical_roi = ['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins']
lh_dlPFC_index = [205, 246, 247, 249, 250, 252, 262, 263, 264, 265, 266, 276, 277]
rh_dlPFC_index = [25, 66, 67, 69, 70, 72, 82, 83, 84, 85, 86, 96, 97]
lh_mPFC_index = [236, 237, 238, 239, 240, 241, 242, 243, 244, 248, 267, 343, 344, 345, 358, 359]
rh_mPFC_index = [56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 87, 163, 164, 165, 178, 179]
lh_PCC_index = [193, 194, 206, 209, 210, 211, 212, 213, 214, 300, 321, 340, 341]
rh_PCC_index = [13, 14, 26, 29, 30, 31, 32, 33, 34, 120, 141, 160, 161]
lh_Ins_index = [285, 287, 288, 289, 290, 291, 293, 294, 346, 347, 348, 357]
rh_Ins_index = [105, 107, 108, 109, 110, 111, 113, 114, 166, 167, 168, 177]
dic_cortical_roi = {
    'lh_dlPFC': lh_dlPFC_index,
    'rh_dlPFC': rh_dlPFC_index,
    'lh_mPFC': lh_mPFC_index,
    'rh_mPFC': rh_mPFC_index,
    'lh_PCC': lh_PCC_index,
    'rh_PCC': rh_PCC_index,
    'lh_Ins': lh_Ins_index,
    'rh_Ins': rh_Ins_index
}

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure


# === Step 1: Define Functions === #

def load_dataset(base_dir, data_set):
    """
    Load dataset-specific files.
    """
    fMRIinfo_file_path = os.path.join(base_dir, f"{data_set}_data_set.csv")
    participant_file_path = os.path.join(base_dir, "participants_fMRI.csv")
    return pd.read_csv(fMRIinfo_file_path), pd.read_csv(participant_file_path)


def load_subject_timeseries(subject_ID, session_ID, derivatives_dir):
    """
    Load cortical and subcortical timeseries data for a subject.
    """
    cortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, "timeseries", cortical_file_name)

    subcortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, "timeseries", subcortical_file_name)

    if not (os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path)):
        print(f"Missing files for subject {subject_ID}, session {session_ID}.")
        return None

    # Load and process cortical timeseries
    df_cortical_all = pd.read_csv(cortical_file_path, compression="gzip", index_col=0, header=0)

    # Load and process subcortical timeseries
    df_subcortical_all = pd.read_csv(subcortical_file_path, compression="gzip", index_col=0, header=0)

    # Combine cortical and subcortical ROIs
    return pd.concat([df_cortical_all.transpose(), df_subcortical_all.transpose()], axis=1)

def add_noise_to_constant_features(data, noise_level=0.01):
    """
    Add Gaussian noise to constant features (std = 0) instead of removing them,
    and replace them in the data matrix.
    
    Parameters:
    - data: numpy.ndarray
        Input data matrix (rows: observations, columns: features).
    - noise_level: float
        Standard deviation of the Gaussian noise to add to constant features.
    
    Returns:
    - data_with_noise: numpy.ndarray
        Data matrix with constant features replaced by their noised versions.
    """
    # Identify constant features (standard deviation = 0 across rows)
    constant_features = np.where(np.std(data, axis=0) == 0)[0]
    
    # Log constant feature indices
    print(f"Identifying constant features to replace with noise at indices: {constant_features}")

    # Generate Gaussian noise for constant features
    noisy_data = data.copy()
    if constant_features.size > 0:
        noisy_data[:, constant_features] = np.random.normal(0, noise_level, 
                                                               size=(data.shape[0], len(constant_features)))
    
    return noisy_data

def clean_data(data):
    """
    Handle missing values and replace constant features with noise.
    This ensures there are no NaNs left in the data.
    """
    # Handle missing values (replace NaNs with column mean)
    data_filled = np.copy(data)
    for j in range(data.shape[1]):
        if np.isnan(data[:, j]).any():
            column_mean = np.nanmean(data[:, j])  # Compute column mean
            if not np.isnan(column_mean):  # Ensure it's a valid mean
                data_filled[:, j] = np.nan_to_num(data[:, j], nan=column_mean)

    # Add noise to constant features
    #data_filled_noisy = add_noise_to_constant_features(data_filled, noise_level=0.05)
    data_filled_noisy = data_filled
    # Verify that no NaNs remain
    # if np.isnan(data_filled_noisy).any():
    #     print("Warning: NaNs still exist in data after cleaning.")
    # else:
    #     print("No NaNs present after cleaning.")

    return data_filled_noisy


def compute_connectivity(data):
    """
    Compute connectivity matrix for the given time series data.
    """
    correlation_measure = ConnectivityMeasure(kind="correlation")
    return correlation_measure.fit_transform([data])[0]


def extract_upper_triangle(matrix):
    """
    Extract the upper triangle values (excluding diagonal) from a connectivity matrix.
    """
    upper_tri_indices = np.triu_indices(matrix.shape[0], k=1)
    return matrix[upper_tri_indices]


# === Step 2: Define the Pipeline === #

def process_fMRI_subject(subject_ID, session_ID, derivatives_dir):
    """
    Full pipeline for processing a single subject's fMRI data.
    """
    # Load subject timeseries
    df_roi = load_subject_timeseries(subject_ID, session_ID, derivatives_dir)
    if df_roi is None:
        return None, None

    # Clean data
    cleaned_data = clean_data(df_roi.values)
    # Check NaNs again before standardization
    if np.isnan(cleaned_data).any():
        print(f"Error: NaNs found in cleaned data for subject {subject_ID}")
        return None, None
    # Standardize data
    #standardized_data = StandardScaler().fit_transform(df_roi.values)
    standardized_data = StandardScaler().fit_transform(cleaned_data)

    # Compute connectivity matrix
    connectivity_matrix = compute_connectivity(standardized_data)

    # Extract upper triangle
    upper_triangle = extract_upper_triangle(connectivity_matrix)

    return upper_triangle, subject_ID


def process_fMRI_data(data_set, user_dir, project_name, session_ID):
    """
    Full pipeline for processing fMRI data for all subjects.
    """
    # Set paths
    base_dir = os.path.join(user_dir, project_name, "data")
    derivatives_dir = os.path.join(base_dir, "derivatives")

    # Load dataset
    df_fMRIinfo, df_participants = load_dataset(base_dir, data_set)
    subject_IDs = df_fMRIinfo["eid"].unique()

    # Initialize lists for data
    connectivity_data = []
    subject_ids_cleaned = []

    # Process each subject individually
    for subject_ID in subject_IDs:
        upper_triangle, cleaned_subject_ID = process_fMRI_subject(
            subject_ID, session_ID, derivatives_dir
        )
        if upper_triangle is not None:
            connectivity_data.append(upper_triangle)
            subject_ids_cleaned.append(cleaned_subject_ID)

    # Filter participants based on available data
    df_filtered = df_participants.loc[df_participants["eid"].isin(subject_ids_cleaned)]

    return np.array(connectivity_data), df_filtered


In [3]:
#codes for modeling
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Ensure binary target
def ensure_binary_target(y):
    unique_values = np.unique(y)
    if len(unique_values) > 2:
        raise ValueError("Target variable contains more than two classes. Please preprocess the data.")
    if unique_values.dtype == bool:
        return y.astype(int)
    elif set(unique_values) == {0, 1} or set(unique_values) == {1, 0}:
        return y
    else:
        raise ValueError("Target variable is not binary. Please preprocess the data.")


# Split data into training and testing sets
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)


# Model selection using cross-validation
def model_selection(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),  # Provides coefficients (coef_)
        "Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),  # coef_
        "Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),  # coef_
        "LDA": LinearDiscriminantAnalysis(),  # Provides coefficients (coef_)
        "Perceptron": Perceptron(),  # Provides coefficients (coef_)
        "SVM (Linear)": SVC(kernel='linear'),  # Provides coefficients (coef_) when kernel='linear'
        "Random Forest": RandomForestClassifier(),  # Provides feature_importances_
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    for model_name, model in models.items():
        cv_score = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy').mean()
        print(f"Model: {model_name}, CV Score: {cv_score:.4f}")

        if cv_score > best_score:
            best_score = cv_score
            best_model = model
            best_name = model_name

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model, best_name


from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

def feature_selection_with_rfe(X_train, y_train, n_features, best_model):
    """
    Perform feature selection using RFE, with fallback to univariate selection for models without coefficients.
    """
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        # Use RFE for models with coefficients or feature importances
        rfe = RFE(estimator=best_model, n_features_to_select=n_features, step=1)
        rfe.fit(X_train, y_train)

        selected_features = rfe.support_
        if np.sum(selected_features) == 0:
            print("No features selected using RFE. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    else:
        # Fallback to univariate feature selection
        print("Model lacks coefficients/feature importance; using univariate feature selection.")
        
        # Use SelectKBest with F-statistic (or mutual information if preferred)
        selector = SelectKBest(score_func=f_classif, k=n_features)
        selector.fit(X_train, y_train)

        selected_features = selector.get_support()
        if np.sum(selected_features) == 0:
            print("No features selected using univariate method. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    return selected_features

from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import numpy as np

def feature_selection_with_rfe_cv(X_train, y_train, best_model, scoring_metric='accuracy'):
    """
    Perform feature selection using RFE or univariate selection, optimizing the number of features automatically
    using cross-validation.
    """
    
    def evaluate_features(model, X, y, num_features):
        """
        Helper function to evaluate the model's performance with the given number of features using cross-validation.
        """
        if hasattr(model, "coef_") or hasattr(model, "feature_importances_"):
            # Perform RFE with the given number of features
            rfe = RFE(estimator=model, n_features_to_select=num_features, step=1)
            X_selected = rfe.fit_transform(X, y)
        else:
            # Use univariate feature selection as a fallback
            selector = SelectKBest(score_func=f_classif, k=num_features)
            X_selected = selector.fit_transform(X, y)

        # Evaluate model performance using cross-validation
        scores = cross_val_score(model, X_selected, y, cv=5, scoring=scoring_metric)
        return scores.mean()

    # Iterate over a range of features to find the optimal number of features
    best_score = -np.inf
    optimal_num_features = 0
    for num_features in range(1, X_train.shape[1] + 1):
        score = evaluate_features(best_model, X_train, y_train, num_features)
        if score > best_score:
            best_score = score
            optimal_num_features = num_features

    print(f"Optimal number of features: {optimal_num_features} with cross-validated score: {best_score:.4f}")

    # Perform final RFE or univariate selection with the optimal number of features
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        rfe = RFE(estimator=best_model, n_features_to_select=optimal_num_features, step=1)
        rfe.fit(X_train, y_train)
        selected_features = rfe.support_
    else:
        selector = SelectKBest(score_func=f_classif, k=optimal_num_features)
        selector.fit(X_train, y_train)
        selected_features = selector.get_support()

    return selected_features, optimal_num_features


# Two-step grid search for hyperparameter optimization
def tune_model_hyperparameters(model, model_name, X_train, y_train):
    refined_grid = {}  # Initialize with a default value to avoid "unbound variable" error

    if model_name == "Logistic Regression":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Ridge Classifier":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Lasso (L1)" or model_name == "ElasticNet (L1+L2)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "SVM (Linear)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Random Forest":
        broad_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
    elif model_name == "Perceptron":
        broad_param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
    elif model_name == "LDA":
        broad_param_grid = {'shrinkage': [None, 'auto'], 'solver': ['svd', 'lsqr', 'eigen']}
    else:
        raise ValueError(f"Model {model_name} does not have a defined parameter grid.")

    # Broad Grid Search
    broad_search = GridSearchCV(model, broad_param_grid, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
    broad_search.fit(X_train, y_train)
    best_params_broad = broad_search.best_params_

    # Define refined grid based on broad search results
    if model_name in ["Logistic Regression", "Lasso (L1)", "ElasticNet (L1+L2)", "SVM (Linear)"]:
        refined_grid = {'C': np.linspace(best_params_broad['C'] * 0.1, best_params_broad['C'] * 10, 5)}
    elif model_name == "Random Forest":
        refined_grid = {
            'n_estimators': [max(10, best_params_broad['n_estimators'] - 50), best_params_broad['n_estimators'], best_params_broad['n_estimators'] + 50],
            'max_depth': [None] if not best_params_broad['max_depth'] else [
                max(1, best_params_broad['max_depth'] - 5), best_params_broad['max_depth'], best_params_broad['max_depth'] + 5]
        }
    elif model_name == "Perceptron":
        refined_grid = {'alpha': np.linspace(best_params_broad['alpha'] * 0.1, best_params_broad['alpha'] * 10, 5)}
    elif model_name == "LDA":
        refined_grid = {'shrinkage': [best_params_broad['shrinkage']], 'solver': [best_params_broad['solver']]}

    # Narrow Grid Search
    narrow_search = GridSearchCV(model, refined_grid, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
    narrow_search.fit(X_train, y_train)

    print(f"Best Parameters (Broad Search): {best_params_broad}")
    print(f"Best Parameters (Narrow Search): {narrow_search.best_params_}")

    best_model = narrow_search.best_estimator_
    return best_model


from sklearn.metrics import roc_auc_score
# Train and evaluate final model
def train_and_evaluate_final_model(X_train, y_train, X_test, y_test, model):
    """
    Train and evaluate the final model. Reports accuracy and AUC.
    
    Parameters:
    - X_train: Features for training.
    - y_train: Labels for training.
    - X_test: Features for testing.
    - y_test: Labels for testing.
    - model: Machine learning model (must support `fit` and `predict_proba`).

    Returns:
    - model: Trained model.
    - test_accuracy: Accuracy on the test set.
    - test_auc: AUC score on the test set.
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate accuracy
    test_accuracy = model.score(X_test, y_test)
    
    # Predict probabilities for AUC computation
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
        test_auc = roc_auc_score(y_test, y_proba)
        print(f"Test Set AUC (Final Model): {test_auc:.4f}")
    else:
        print("Model does not support probability predictions; skipping AUC computation.")
        test_auc = None

    print(f"Test Set Accuracy (Final Model): {test_accuracy:.4f}")
    return model, test_accuracy, test_auc



# Cosine similarity between two models
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_model_similarity(model1, model2):
    """
    Calculate the similarity between two models using cosine similarity or feature importances.

    Parameters:
    - model1: First trained model
    - model2: Second trained model

    Returns:
    - similarity: Cosine similarity score between the two models' coefficients or importances.
    """
    # Extract the weights (coefficients) or feature importances
    def get_model_vector(model):
        if hasattr(model, 'coef_'):  # Linear models with coefficients
            return model.coef_.flatten()
        elif hasattr(model, 'feature_importances_'):  # Tree-based models
            return model.feature_importances_
        else:
            raise ValueError(f"Model of type {type(model)} does not have coefficients or feature importances.")
    
    try:
        # Get vectors for the two models
        vector1 = get_model_vector(model1)
        vector2 = get_model_vector(model2)

        # Ensure vectors are of the same length
        if len(vector1) != len(vector2):
            raise ValueError("Model vectors have different lengths. Ensure the models were trained on the same features.")

        # Calculate cosine similarity
        similarity = cosine_similarity([vector1], [vector2])
        return similarity[0][0]  # Return the scalar similarity value

    except ValueError as e:
        print(f"Error in calculating similarity: {e}")
        return None


# Full pipeline
def pipeline1(X, y, n_features):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y)

    best_model, best_name = model_selection(X_train, y_train)

    selected_features = feature_selection_with_rfe(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy

def pipeline2(X, y):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y)

    best_model, best_name = model_selection(X_train, y_train)

    
    selected_features, features_number = feature_selection_with_rfe_cv(X_train, y_train, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy

# Compare CAD and PAD models
def compare_models_and_analyze_topography1(X_data_set1, y_data_set1, X_data_set2, y_data_set2, n_features):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline1(X_data_set1, y_data_set1, n_features)
    X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    y_pred = final_model_data_set1.predict(X_data_set1_selected)
    print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
    print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline1(X_data_set2, y_data_set2, n_features)
    X_data_set2_selected = X_data_set2[:, selected_features_data_set2]
    y_pred2 = final_model_data_set2.predict(X_data_set2_selected)
    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred2))
    

    print("\nEvaluating data_set1 model on data_set2 dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity

def compare_models_and_analyze_topography2(X_data_set1, y_data_set1, X_data_set2, y_data_set2):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline2(X_data_set1, y_data_set1)
    X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    y_pred = final_model_data_set1.predict(X_data_set1_selected)
    print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
    print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline2(X_data_set2, y_data_set2)
    X_data_set2_selected = X_data_set2[:, selected_features_data_set2]
    y_pred2 = final_model_data_set2.predict(X_data_set2_selected)
    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred2))
    

    print("\nEvaluating data_set1 model on data_set2 dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity

In [4]:
# Define user inputs
user_dir = "/Users/xiaoqianxiao"
project_name = "UKB"
session_ID = 2  # Specify session

In [5]:
# Run the pipeline
data_set = "past_anxiety"  # Dataset identifier
X_pad, df_PAD = process_fMRI_data(data_set, user_dir, project_name, session_ID)
y_pad = df_PAD["hospital_not_now"]
y_pad_GAD7 = df_PAD["GAD7_score"]

data_set = "current_anxiety"  # Dataset identifier
X_cad, df_CAD = process_fMRI_data(data_set, user_dir, project_name, session_ID)
y_cad = df_CAD["hospital_current_anxiety"]
y_cad_GAD7 = df_CAD["GAD7_score"]

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 4556454


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 1630706


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 2149874


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 4393701


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 3936293


In [7]:
print(f'n(cad)/(cad_control): {sum(y_cad==True)}/{sum(y_cad==False)} = {((sum(y_cad==True))/(sum(y_cad==False))):.2f}')
print(f'n(pad)/(pad_control): {sum(y_pad==True)}/{sum(y_pad==False)} = {((sum(y_pad==True))/(sum(y_pad==False))):.2f}')

n(cad)/(cad_control): 118/104 = 1.13
n(pad)/(pad_control): 510/476 = 1.07


In [8]:
n_features = 10
compare_models_and_analyze_topography1(X_cad, y_cad, X_pad, y_pad, n_features)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5072
Model: Ridge Classifier, CV Score: 0.5127


KeyboardInterrupt: 

In [ ]:
compare_models_and_analyze_topography2(X_cad, y_cad, X_pad, y_pad)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5072
Model: Ridge Classifier, CV Score: 0.5127
Model: Lasso (L1), CV Score: 0.4471
Model: LDA, CV Score: 0.4359
Model: Perceptron, CV Score: 0.5085
Model: SVM (Linear), CV Score: 0.4961
Model: Random Forest, CV Score: 0.5255
Best Model: Random Forest with CV score: 0.5255


In [9]:
data_set = "ah"  # Dataset identifier
X_ah, df_ah = process_fMRI_data(data_set, user_dir, project_name, session_ID)
y_ah = df_ah["active_history"]

data_set = "ih"  # Dataset identifier
X_ih, df_ih = process_fMRI_data(data_set, user_dir, project_name, session_ID)
y_ih = df_ih["inactive_history"]

data_set = "a_noh"  # Dataset identifier
X_a_noh, df_a_noh = process_fMRI_data(data_set, user_dir, project_name, session_ID)
y_a_noh = df_a_noh["active_no_history"]

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 4642328


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_55750/49930889.py:80: RuntimeWarning: Mean of empty slice
  column_mean = np.nanmean(data[:, j])  # Compute column mean


Error: NaNs found in cleaned data for subject 2211729
Missing files for subject 1529291, session 2.


In [61]:
print(f'n(ah)/(ah_control): {sum(y_ah==True)}/{sum(y_ah==False)} = {((sum(y_ah==True))/(sum(y_ah==False))):.2f}')
print(f'n(ih)/(ih_control): {sum(y_ih==True)}/{sum(y_ih==False)} = {((sum(y_ih==True))/(sum(y_ih==False))):.2f}')
print(f'n(a_noh)/(a_noh_control): {sum(y_a_noh==True)}/{sum(y_a_noh==False)} = {((sum(y_a_noh==True))/(sum(y_a_noh==False))):.2f}')

n(ah)/(ah_control): 366/337 = 1.09
n(ih)/(ih_control): 873/819 = 1.07
n(a_noh)/(a_noh_control): 250/233 = 1.07


In [62]:
n_features = 20
compare_models_and_analyze_topography1(X_ah, y_ah, X_ih, y_ih, n_features)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5268
Model: Ridge Classifier, CV Score: 0.5196
Model: Lasso (L1), CV Score: 0.4803
Model: LDA, CV Score: 0.5413
Model: Perceptron, CV Score: 0.5161
Model: SVM (Linear), CV Score: 0.5054
Model: Random Forest, CV Score: 0.5231
Best Model: LDA with CV score: 0.5413
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 6 candidates, totalling 60 fits


/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/base.py", line 1473, in

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best Parameters (Broad Search): {'shrinkage': 'auto', 'solver': 'lsqr'}
Best Parameters (Narrow Search): {'shrinkage': 'auto', 'solver': 'lsqr'}
Test Set AUC (Final Model): 0.4612
Test Set Accuracy (Final Model): 0.4681
Confusion Matrix:
 [[198 139]
 [123 243]]
Classification Report:
               precision    recall  f1-score   support

       False       0.62      0.59      0.60       337
        True       0.64      0.66      0.65       366

    accuracy                           0.63       703
   macro avg       0.63      0.63      0.63       703
weighted avg       0.63      0.63      0.63       703


Training data_set2 model...
Model: Logistic Regression, CV Score: 0.5107
Model: Ridge Classifier, CV Score: 0.5062
Model: Lasso (L1), CV Score: 0.4997
Model: LDA, CV Score: 0.4973
Model: Perceptron, CV Score: 0.5004
Model: SVM (Linear), CV Score: 0.4980


KeyboardInterrupt: 

In [516]:
compare_models_and_analyze_topography2(X_ah, y_ah, X_ih, y_ih)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5383
Model: Ridge Classifier, CV Score: 0.5383
Model: Lasso (L1), CV Score: 0.5311
Model: ElasticNet (L1+L2), CV Score: 0.5223
Model: LDA, CV Score: 0.4956
Model: Perceptron, CV Score: 0.5363
Model: SVM (Linear), CV Score: 0.5242
Model: Random Forest, CV Score: 0.5188
Best Model: Logistic Regression with CV score: 0.5383
Optimal number of features: 3 with cross-validated score: 0.5934
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 100}
Best Parameters (Narrow Search): {'C': np.float64(257.5)}
Test Set AUC (Final Model): 0.5733
Test Set Accuracy (Final Model): 0.5177
Confusion Matrix:
 [[154 183]
 [107 260]]
Classification Report:
               precision    recall  f1-score   support

       False       0.59      0.46      0.52       337
        True       0.59      0.71      0.64       367

    accuracy    

In [65]:
n_features = 20
compare_models_and_analyze_topography1(X_ah, y_ah, X_a_noh, y_a_noh, n_features)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5195
Model: Ridge Classifier, CV Score: 0.5106
Model: Lasso (L1), CV Score: 0.5233
Model: LDA, CV Score: 0.4841
Model: Perceptron, CV Score: 0.4966
Model: SVM (Linear), CV Score: 0.5017
Model: Random Forest, CV Score: 0.4984
Best Model: Lasso (L1) with CV score: 0.5233
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 1}
Best Parameters (Narrow Search): {'C': np.float64(2.575)}
Test Set AUC (Final Model): 0.4613
Test Set Accuracy (Final Model): 0.4539
Confusion Matrix:
 [[197 140]
 [113 253]]
Classification Report:
               precision    recall  f1-score   support

       False       0.64      0.58      0.61       337
        True       0.64      0.69      0.67       366

    accuracy                           0.64       703
 

/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/base.py", line 1473, in

np.float64(0.2562064289986569)

In [517]:
compare_models_and_analyze_topography2(X_ah, y_ah, X_a_noh, y_a_noh)

Training data_set1 model...
Model: Logistic Regression, CV Score: 0.5383
Model: Ridge Classifier, CV Score: 0.5383
Model: Lasso (L1), CV Score: 0.5311
Model: ElasticNet (L1+L2), CV Score: 0.5223
Model: LDA, CV Score: 0.4956
Model: Perceptron, CV Score: 0.5363
Model: SVM (Linear), CV Score: 0.5242
Model: Random Forest, CV Score: 0.4938
Best Model: Logistic Regression with CV score: 0.5383
Optimal number of features: 3 with cross-validated score: 0.5934
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 100}
Best Parameters (Narrow Search): {'C': np.float64(257.5)}
Test Set AUC (Final Model): 0.5733
Test Set Accuracy (Final Model): 0.5177
Confusion Matrix:
 [[154 183]
 [107 260]]
Classification Report:
               precision    recall  f1-score   support

       False       0.59      0.46      0.52       337
        True       0.59      0.71      0.64       367

    accuracy    

/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
 

Best Parameters (Broad Search): {'C': 1}
Best Parameters (Narrow Search): {'C': np.float64(2.575)}
Test Set AUC (Final Model): 0.4091
Test Set Accuracy (Final Model): 0.4433
Confusion Matrix:
 [[154  79]
 [ 97 153]]
Classification Report:
               precision    recall  f1-score   support

       False       0.61      0.66      0.64       233
        True       0.66      0.61      0.63       250

    accuracy                           0.64       483
   macro avg       0.64      0.64      0.64       483
weighted avg       0.64      0.64      0.64       483


Evaluating data_set1 model on data_set2 dataset:
Accuracy of data_set1 model on data_set2 data: 0.5031
Dataset 2 - Accuracy: 0.5031, AUC: 0.4816
Confusion Matrix:
 [[ 97 136]
 [104 146]]
Classification Report:
               precision    recall  f1-score   support

       False       0.48      0.42      0.45       233
        True       0.52      0.58      0.55       250

    accuracy                           0.50       483
   

In [447]:
sum(y_ih==True)/len(y_ih)

0.22388059701492538